In [25]:
# Import of the pyomo module
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt

# Creation of a Concrete Model
model = ConcreteModel()

idx_end = 1000
t_end = 10.0
Dt = t_end/idx_end
model.i = Set(initialize=range(idx_end), doc='tiempo')
model.j = Set(initialize=range(idx_end-1), doc='tiempo reducido')
model.k = Set(initialize=range(2), doc='tiempo reducido')

model.i_l = Var(model.i, bounds=(0.0,None), doc='current')
model.v_ref = Var(model.i, bounds=(-5,5), doc='voltage')
model.v = Var(model.i, bounds=(-5,5), doc='voltage')

def _dv(model, i):
    T_filter = 0.01
    return model.v[i+1] == model.v[i] + Dt*0.5*(1/T_filter*(model.v_ref[i+1]-model.v[i+1])+1/T_filter*(model.v_ref[i]-model.v[i]))
model.veq = Constraint(model.j, rule=_dv, doc='restr')

def _di_l(model, i):
    return model.i_l[i+1] == model.i_l[i] + Dt*0.5*(1/1.0*(model.v[i+1]-0.1*model.i_l[i+1])+1/1.0*(model.v[i]-0.1*model.i_l[i]))
model.ileq = Constraint(model.j, rule=_di_l, doc='restr')

def con_desired_1(model):
    return model.i_l[idx_end/2] == 10
model.desired1 = Constraint(rule=con_desired_1, doc='restr')

def con_desired_2(model):
    return model.i_l[idx_end-1] == 5
model.desired2 = Constraint(rule=con_desired_2, doc='restr')

def initial1(model):
    return model.v[0] == 0
model.initial1 = Constraint(rule=initial1, doc='initial value')

def initial2(model):
    return model.i_l[0] == 0
model.initial2 = Constraint(rule=initial2, doc='initial value')

def objective_rule(model):
    e = 0
    for i in range(100):
        e += model.v[i]*model.i_l[i] + 1000*model.i_l[i]**2
    
    return  e
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')


from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("ipopt")
results = opt.solve(model)

t = np.array([item for item in model.i_l.get_values()])*1.0
i_l = np.array([item for item in model.i_l.get_values().values()])
v_ref = np.array([item for item in model.v_ref.get_values().values()])
v = np.array([item for item in model.v.get_values().values()])

In [26]:
fig, (ax0,ax1) = plt.subplots(nrows=2)   # creates a figure with one axe

ax0.plot(t,i_l)

ax1.plot(t,v_ref, 'r', lw=1)
ax1.plot(t,v, 'g', lw=1)
#ax1.plot(t,v_1)
#ax1.plot(t,v_2)
#ax1.plot(t,v_3)

ax0.grid(True)
ax1.grid(True)

plt.show()


In [ ]:
t

In [ ]:
results